This notebook will take the corpus, translate it into a gensim TaggedDocument list, and train the doc2vec model in order to extract the document vecotrs.  We will then translate that into a pandas data frame and save to a file for use in kmeans clustering.

In [2]:
import pickle
import numpy
from gensim.models import Doc2Vec as d2v
from gensim.models.doc2vec import TaggedDocument as td
import pandas as pd

In [3]:
with open('NHlegtext.corpus','rb') as f:
    legcorpus = pickle.load(f)
#remove legslative stop words (if not accomplished in the legislation retrieval)
#leg_stop_words = ['rep','sen','bold','italics','sponsors']

print(legcorpus[1])

['894913', '1441965', ['concurrent', 'resolution', 'proposing', 'consitutional', 'amendment', 'relating', 'natural', 'rights', 'children', 'providing', 'children', 'natural', 'right', 'protected', 'parents', 'resolved', 'house', 'representatives', 'senate', 'concurring', 'constitution', 'new', 'hampshire', 'amended', 'follows', 'first', 'part', 'constitution', 'amended', 'inserting', 'article', 'following', 'new', 'article', 'art', 'natural', 'rights', 'children', 'minor', 'children', 'natural', 'right', 'protection', 'parents', 'control', 'health', 'education', 'welfare', 'ii', 'amendment', 'proposed', 'constitution', 'submitted', 'qualified', 'voters', 'state', 'state', 'general', 'election', 'held', 'november', 'iii', 'selectmen', 'towns', 'cities', 'wards', 'places', 'state', 'directed', 'insert', 'warrants', 'said', 'election', 'article', 'following', 'effect', 'decide', 'whether', 'amendments', 'constitution', 'proposed', 'session', 'general', 'court', 'shall', 'approved', 'iv', 

In [4]:
# here we'll transform the legcorpus into a "tagged" document for use with doc2vec
# the tags will simply be the document ids.  Since we won't be using the predictor part of the operation, and only the resulting weights,
# we don't need to concern ourselves with labeling the documents
tagged_legdocs = []
for i, l in enumerate(legcorpus):
    if i == 1:
        print(td(l[2], [l[1]]))
    tagged_legdocs.append(td(l[2], [l[1]]))

TaggedDocument(['concurrent', 'resolution', 'proposing', 'consitutional', 'amendment', 'relating', 'natural', 'rights', 'children', 'providing', 'children', 'natural', 'right', 'protected', 'parents', 'resolved', 'house', 'representatives', 'senate', 'concurring', 'constitution', 'new', 'hampshire', 'amended', 'follows', 'first', 'part', 'constitution', 'amended', 'inserting', 'article', 'following', 'new', 'article', 'art', 'natural', 'rights', 'children', 'minor', 'children', 'natural', 'right', 'protection', 'parents', 'control', 'health', 'education', 'welfare', 'ii', 'amendment', 'proposed', 'constitution', 'submitted', 'qualified', 'voters', 'state', 'state', 'general', 'election', 'held', 'november', 'iii', 'selectmen', 'towns', 'cities', 'wards', 'places', 'state', 'directed', 'insert', 'warrants', 'said', 'election', 'article', 'following', 'effect', 'decide', 'whether', 'amendments', 'constitution', 'proposed', 'session', 'general', 'court', 'shall', 'approved', 'iv', 'wordin

In [5]:
embedder = d2v(tagged_legdocs, dm=0, alpha=.025,vector_size=16, min_alpha=.025, min_count=0, epochs=20, workers=4)

In [6]:
embedder.train(tagged_legdocs, total_examples=embedder.corpus_count, epochs=embedder.epochs)

In [14]:
# translate the embeddings and tags into a dataframe
doc_embedding_list = []
for i in range(len(embedder.docvecs)):
    doc_tag = embedder.docvecs.index_to_doctag(i)
    doc_vec = [doc_tag]
    doc_vec.extend(embedder.docvecs[doc_tag].tolist())
    #doc_vec = doc_vec.tolist()
    
    doc_embedding_list.append(doc_vec)

DFlabels = ['doc_id','dv1','dv2','dv3','dv4','dv5'
            ,'dv6','dv7','dv8','dv9','dv10','dv11'
            ,'dv12','dv13','dv14','dv15','dv16'
           # ,'dv17','dv18','dv19','dv20'
           ]

doc_embed_DF = pd.DataFrame(doc_embedding_list, columns=DFlabels)

In [19]:
#added by Ross
#I converted bill_id and doc_id from object to int 64 data types
#so we can merge/join the dataframes
# translate the embeddings and tags into a dataframe
_doc_embedding_list = []
print(len(embedder.docvecs))
print(len(legcorpus))
co = 0


temp_list = []
for i, val in enumerate(range(len(embedder.docvecs))):
    doc_tag = embedder.docvecs.index_to_doctag(i)
    
    doc_vec = [int(doc_tag)]
    doc_vec.extend(embedder.docvecs[doc_tag].tolist())
    tempy = legcorpus[i]
    doc_vec.append(int(tempy[0]))
    #doc_vec = doc_vec.tolist()

    _doc_embedding_list.append(doc_vec)


DFlabels = ['doc_id','dv1','dv2','dv3','dv4','dv5'
            ,'dv6','dv7','dv8','dv9','dv10','dv11'
            ,'dv12','dv13','dv14','dv15','dv16', 'bill_id']

_doc_embed_DF = pd.DataFrame(_doc_embedding_list, columns=DFlabels)

#bill_ids=[]
#for i in legcorpus:
    #append the bill id
#    bill_ids.append(i[0]) 
    
#doc_embed_DF['bill_id'] = bill_ids

#dump data out to a binary file
with open('embedded_pol.DF','wb') as f:
    pickle.dump(_doc_embed_DF,f)

4060
4060


In [15]:
bill_ids=[]
for i in legcorpus:
    #append the bill id
    bill_ids.append(i[0]) 
    
doc_embed_DF['bill_id'] = bill_ids

In [16]:
#dump data out to a binary file
with open('NHlegtext_embed_wBill.DF','wb') as f:
    pickle.dump(doc_embed_DF,f)

In [17]:
#checking for missing values
for i in legcorpus:
    if i[1] == '2030127':
        print('Here!')